In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

import tensorflow as tf

# Parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

# Number of samples to calculate validation and accuracy
# Decrease this if you're running out of memory to calculate accuracy
test_valid_size = 256

# Network Parameters
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # Dropout, probability to keep units


# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}


def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} - Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./t10k-labels-idx1-ubyte.gz
Epoch  1, Batch   1 - Loss: 79263.4609 Validation Accuracy: 0.113281
Epoch  1, Batch   2 - Loss: 57077.1016 Validation Accuracy: 0.132812
Epoch  1, Batch   3 - Loss: 48076.3828 Validation Accuracy: 0.144531
Epoch  1, Batch   4 - Loss: 40412.9258 Validation Accuracy: 0.125000
Epoch  1, Batch   5 - Loss: 38096.7070 Validation Accuracy: 0.117188
Epoch  1, Batch   6 - Loss: 30880.1133 Validation Accuracy: 0.113281
Epoch  1, Batch   7 - Loss: 30414.0273 Validation Accuracy: 0.132812
Epoch  1, Batch   8 - Loss: 30575.7207 Validation Accuracy: 0.109375
Epoch  1, Batch   9 - Loss: 27696

Epoch  1, Batch 116 - Loss:  4589.1738 Validation Accuracy: 0.519531
Epoch  1, Batch 117 - Loss:  4879.9258 Validation Accuracy: 0.515625
Epoch  1, Batch 118 - Loss:  5052.5068 Validation Accuracy: 0.535156
Epoch  1, Batch 119 - Loss:  4880.9072 Validation Accuracy: 0.542969
Epoch  1, Batch 120 - Loss:  5786.0088 Validation Accuracy: 0.542969
Epoch  1, Batch 121 - Loss:  4138.9980 Validation Accuracy: 0.546875
Epoch  1, Batch 122 - Loss:  4943.5728 Validation Accuracy: 0.550781
Epoch  1, Batch 123 - Loss:  4586.2656 Validation Accuracy: 0.554688
Epoch  1, Batch 124 - Loss:  4375.6689 Validation Accuracy: 0.554688
Epoch  1, Batch 125 - Loss:  5557.1475 Validation Accuracy: 0.554688
Epoch  1, Batch 126 - Loss:  4006.1372 Validation Accuracy: 0.550781
Epoch  1, Batch 127 - Loss:  3592.7363 Validation Accuracy: 0.562500
Epoch  1, Batch 128 - Loss:  4583.7168 Validation Accuracy: 0.558594
Epoch  1, Batch 129 - Loss:  4112.1914 Validation Accuracy: 0.554688
Epoch  1, Batch 130 - Loss:  4077.

Epoch  1, Batch 240 - Loss:  2523.5054 Validation Accuracy: 0.671875
Epoch  1, Batch 241 - Loss:  2394.0339 Validation Accuracy: 0.667969
Epoch  1, Batch 242 - Loss:  2646.1973 Validation Accuracy: 0.667969
Epoch  1, Batch 243 - Loss:  2748.2637 Validation Accuracy: 0.667969
Epoch  1, Batch 244 - Loss:  2490.7412 Validation Accuracy: 0.664062
Epoch  1, Batch 245 - Loss:  2161.3367 Validation Accuracy: 0.671875
Epoch  1, Batch 246 - Loss:  2626.8357 Validation Accuracy: 0.671875
Epoch  1, Batch 247 - Loss:  2192.6675 Validation Accuracy: 0.671875
Epoch  1, Batch 248 - Loss:  2230.1753 Validation Accuracy: 0.679688
Epoch  1, Batch 249 - Loss:  1845.6934 Validation Accuracy: 0.675781
Epoch  1, Batch 250 - Loss:  2153.6755 Validation Accuracy: 0.683594
Epoch  1, Batch 251 - Loss:  1974.5961 Validation Accuracy: 0.683594
Epoch  1, Batch 252 - Loss:  2384.7327 Validation Accuracy: 0.687500
Epoch  1, Batch 253 - Loss:  2935.7715 Validation Accuracy: 0.683594
Epoch  1, Batch 254 - Loss:  2412.

Epoch  1, Batch 363 - Loss:  1540.8037 Validation Accuracy: 0.710938
Epoch  1, Batch 364 - Loss:  1965.7820 Validation Accuracy: 0.714844
Epoch  1, Batch 365 - Loss:  1711.0771 Validation Accuracy: 0.718750
Epoch  1, Batch 366 - Loss:  2316.7961 Validation Accuracy: 0.718750
Epoch  1, Batch 367 - Loss:  1637.7019 Validation Accuracy: 0.714844
Epoch  1, Batch 368 - Loss:  1760.1310 Validation Accuracy: 0.718750
Epoch  1, Batch 369 - Loss:  1935.6802 Validation Accuracy: 0.714844
Epoch  1, Batch 370 - Loss:  2193.8833 Validation Accuracy: 0.714844
Epoch  1, Batch 371 - Loss:  1500.4449 Validation Accuracy: 0.710938
Epoch  1, Batch 372 - Loss:  1205.6187 Validation Accuracy: 0.703125
Epoch  1, Batch 373 - Loss:  1409.4814 Validation Accuracy: 0.703125
Epoch  1, Batch 374 - Loss:  1758.0674 Validation Accuracy: 0.699219
Epoch  1, Batch 375 - Loss:  2104.5466 Validation Accuracy: 0.707031
Epoch  1, Batch 376 - Loss:  1857.8352 Validation Accuracy: 0.703125
Epoch  1, Batch 377 - Loss:  1257.

Epoch  2, Batch  55 - Loss:  1225.2081 Validation Accuracy: 0.742188
Epoch  2, Batch  56 - Loss:  1315.5870 Validation Accuracy: 0.738281
Epoch  2, Batch  57 - Loss:  1302.2374 Validation Accuracy: 0.738281
Epoch  2, Batch  58 - Loss:  1304.7141 Validation Accuracy: 0.742188
Epoch  2, Batch  59 - Loss:  1141.2914 Validation Accuracy: 0.746094
Epoch  2, Batch  60 - Loss:  1117.8273 Validation Accuracy: 0.738281
Epoch  2, Batch  61 - Loss:  1340.3127 Validation Accuracy: 0.738281
Epoch  2, Batch  62 - Loss:  1361.6646 Validation Accuracy: 0.738281
Epoch  2, Batch  63 - Loss:  1141.1965 Validation Accuracy: 0.738281
Epoch  2, Batch  64 - Loss:  1885.3960 Validation Accuracy: 0.734375
Epoch  2, Batch  65 - Loss:  1280.0078 Validation Accuracy: 0.730469
Epoch  2, Batch  66 - Loss:   954.9272 Validation Accuracy: 0.734375
Epoch  2, Batch  67 - Loss:  1269.0977 Validation Accuracy: 0.738281
Epoch  2, Batch  68 - Loss:  1172.7220 Validation Accuracy: 0.734375
Epoch  2, Batch  69 - Loss:  1485.

...

Epoch  9, Batch 247 - Loss:   179.0566 Validation Accuracy: 0.851562
Epoch  9, Batch 248 - Loss:   253.5275 Validation Accuracy: 0.855469
Epoch  9, Batch 249 - Loss:   230.6721 Validation Accuracy: 0.855469
Epoch  9, Batch 250 - Loss:   125.7779 Validation Accuracy: 0.859375
Epoch  9, Batch 251 - Loss:   265.5078 Validation Accuracy: 0.855469
Epoch  9, Batch 252 - Loss:   243.5641 Validation Accuracy: 0.859375
Epoch  9, Batch 253 - Loss:   213.9240 Validation Accuracy: 0.855469
Epoch  9, Batch 254 - Loss:   274.6900 Validation Accuracy: 0.851562
Epoch  9, Batch 255 - Loss:   242.2350 Validation Accuracy: 0.851562
Epoch  9, Batch 256 - Loss:   144.4843 Validation Accuracy: 0.855469
Epoch  9, Batch 257 - Loss:   271.6984 Validation Accuracy: 0.847656
Epoch  9, Batch 258 - Loss:   172.6417 Validation Accuracy: 0.851562
Epoch  9, Batch 259 - Loss:   244.7114 Validation Accuracy: 0.851562
Epoch  9, Batch 260 - Loss:   127.4663 Validation Accuracy: 0.859375
Epoch  9, Batch 261 - Loss:   297.

Epoch  9, Batch 371 - Loss:   344.8299 Validation Accuracy: 0.863281
Epoch  9, Batch 372 - Loss:   112.0699 Validation Accuracy: 0.855469
Epoch  9, Batch 373 - Loss:   143.3076 Validation Accuracy: 0.855469
Epoch  9, Batch 374 - Loss:   229.1416 Validation Accuracy: 0.851562
Epoch  9, Batch 375 - Loss:   209.8206 Validation Accuracy: 0.851562
Epoch  9, Batch 376 - Loss:   160.6637 Validation Accuracy: 0.859375
Epoch  9, Batch 377 - Loss:   350.2217 Validation Accuracy: 0.859375
Epoch  9, Batch 378 - Loss:   231.5258 Validation Accuracy: 0.855469
Epoch  9, Batch 379 - Loss:   221.3777 Validation Accuracy: 0.855469
Epoch  9, Batch 380 - Loss:   336.0758 Validation Accuracy: 0.851562
Epoch  9, Batch 381 - Loss:   122.4563 Validation Accuracy: 0.855469
Epoch  9, Batch 382 - Loss:   261.6511 Validation Accuracy: 0.859375
Epoch  9, Batch 383 - Loss:   317.3283 Validation Accuracy: 0.859375
Epoch  9, Batch 384 - Loss:   285.7458 Validation Accuracy: 0.859375
Epoch  9, Batch 385 - Loss:   207.

Epoch 10, Batch  62 - Loss:   241.2208 Validation Accuracy: 0.855469
Epoch 10, Batch  63 - Loss:   368.1098 Validation Accuracy: 0.855469
Epoch 10, Batch  64 - Loss:   135.2237 Validation Accuracy: 0.855469
Epoch 10, Batch  65 - Loss:   257.1418 Validation Accuracy: 0.855469
Epoch 10, Batch  66 - Loss:   235.6298 Validation Accuracy: 0.855469
Epoch 10, Batch  67 - Loss:   281.7544 Validation Accuracy: 0.855469
Epoch 10, Batch  68 - Loss:   152.5317 Validation Accuracy: 0.855469
Epoch 10, Batch  69 - Loss:   242.6572 Validation Accuracy: 0.859375
Epoch 10, Batch  70 - Loss:   187.5696 Validation Accuracy: 0.859375
Epoch 10, Batch  71 - Loss:   230.1201 Validation Accuracy: 0.855469
Epoch 10, Batch  72 - Loss:   139.1793 Validation Accuracy: 0.859375
Epoch 10, Batch  73 - Loss:   234.6622 Validation Accuracy: 0.859375
Epoch 10, Batch  74 - Loss:   159.1657 Validation Accuracy: 0.859375
Epoch 10, Batch  75 - Loss:   156.5201 Validation Accuracy: 0.855469
Epoch 10, Batch  76 - Loss:   230.

Epoch 10, Batch 187 - Loss:   194.6559 Validation Accuracy: 0.863281
Epoch 10, Batch 188 - Loss:   162.9717 Validation Accuracy: 0.863281
Epoch 10, Batch 189 - Loss:   252.8443 Validation Accuracy: 0.863281
Epoch 10, Batch 190 - Loss:   240.1779 Validation Accuracy: 0.863281
Epoch 10, Batch 191 - Loss:   219.3786 Validation Accuracy: 0.859375
Epoch 10, Batch 192 - Loss:   209.4279 Validation Accuracy: 0.859375
Epoch 10, Batch 193 - Loss:   203.1700 Validation Accuracy: 0.859375
Epoch 10, Batch 194 - Loss:   193.8890 Validation Accuracy: 0.859375
Epoch 10, Batch 195 - Loss:   332.3723 Validation Accuracy: 0.859375
Epoch 10, Batch 196 - Loss:   160.6411 Validation Accuracy: 0.859375
Epoch 10, Batch 197 - Loss:   155.0008 Validation Accuracy: 0.855469
Epoch 10, Batch 198 - Loss:   338.4785 Validation Accuracy: 0.859375
Epoch 10, Batch 199 - Loss:   394.2805 Validation Accuracy: 0.851562
Epoch 10, Batch 200 - Loss:    98.0029 Validation Accuracy: 0.855469
Epoch 10, Batch 201 - Loss:   184.

Epoch 10, Batch 310 - Loss:   243.1063 Validation Accuracy: 0.859375
Epoch 10, Batch 311 - Loss:   339.8607 Validation Accuracy: 0.867188
Epoch 10, Batch 312 - Loss:   232.3472 Validation Accuracy: 0.863281
Epoch 10, Batch 313 - Loss:   163.2442 Validation Accuracy: 0.867188
Epoch 10, Batch 314 - Loss:   371.6069 Validation Accuracy: 0.863281
Epoch 10, Batch 315 - Loss:   171.6424 Validation Accuracy: 0.863281
Epoch 10, Batch 316 - Loss:   121.5242 Validation Accuracy: 0.859375
Epoch 10, Batch 317 - Loss:   202.0312 Validation Accuracy: 0.863281
Epoch 10, Batch 318 - Loss:   247.4899 Validation Accuracy: 0.867188
Epoch 10, Batch 319 - Loss:   191.4084 Validation Accuracy: 0.859375
Epoch 10, Batch 320 - Loss:   350.9296 Validation Accuracy: 0.855469
Epoch 10, Batch 321 - Loss:   186.6868 Validation Accuracy: 0.863281
Epoch 10, Batch 322 - Loss:   156.7981 Validation Accuracy: 0.851562
Epoch 10, Batch 323 - Loss:   172.4567 Validation Accuracy: 0.843750
Epoch 10, Batch 324 - Loss:   303.